In [ ]:
//*[@id="ChdDSUhNMG9nS0VJQ0FnSURMajZ1d3l3RRAB"]/span[2]/button

In [63]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 구글 맵 페이지 열기 및 '뱅뱅막국수' 검색
driver.get("https://www.google.com/maps")
try:
    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)
    time.sleep(5)  # 검색 후 페이지 로딩 대기

except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")

try:
    # 리뷰 페이지로 이동
    next_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]'))
    )
    next_button.click()
    time.sleep(5)  # 리뷰 페이지 로딩 대기
    print('리뷰로 넘어감')

except (NoSuchElementException, TimeoutException) as e:
    print(f"요소를 찾을 수 없거나 클릭할 수 없습니다: {e}")

# 스크롤 다운하여 모든 리뷰 로드
def scroll_down():
    scrollable_div = None
    for _ in range(10):  # 시도 횟수 조정 가능
        try:
            scrollable_div = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
            )
            break
        except:
            time.sleep(2)

    if scrollable_div is None:
        print("스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.")
        return

    # 스크롤을 끝까지 반복 수행
    last_review_count = len(driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium'))
    for _ in range(30):  # 필요한 만큼 스크롤 반복
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)
        
        new_review_count = len(driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium'))
        if new_review_count == last_review_count:
            break  # 리뷰가 더 이상 로드되지 않으면 종료
        last_review_count = new_review_count

scroll_down()

# 리뷰 데이터 수집
reviews_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
for review in reviews:
    try:
        # '자세히' 버튼 클릭
        try:
            more_button = review.find_element(By.XPATH, './/span[2]/button')
            more_button.click()
            time.sleep(1)  # '자세히' 클릭 후 리뷰 로드 대기
        except NoSuchElementException:
            pass  # '자세히' 버튼이 없으면 무시하고 진행
        
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
        reviews_list.append({
            "ID": reviewer_id,
            "Date": date,
            "Review": review_text
        })
    except Exception as e:
        print(f"리뷰 수집 중 오류: {e}")
        continue

# 데이터프레임으로 변환 및 출력
reviews_df = pd.DataFrame(reviews_list)
print(reviews_df)

# # CSV 파일로 저장
# reviews_df.to_csv('bangbang_makguksu_reviews.csv', index=False, encoding='utf-8-sig')

# # 드라이버 종료
# driver.quit()


리뷰로 넘어감
스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.
          ID  Date                                             Review
0      Amy J  2주 전  막국수의 대가🌟🌟🌟🌟🌟\n배가 작은거에 한탄하며 모든 음식을 먹지 못해 슬플 정도....
1  Sujin Kim  3달 전  들기름막국수, 참기름막국수, 코다리막국수를 먹었는데 3개 메뉴 모두 맛있었어요. 국...
2        정의진  4달 전  유명한 곳이라 와봤는데 맛있고 친절하고 유명한 이유를 알겠네요^-^\n코다리랑 참기...
3  Sunny Lee  4달 전  강남 뱅뱅사거리에서 막국수로 유명한 맛집입니다. 들기름막국수 강추해요. 물막국수, ...
4     schnee  4달 전  여긴 진짜 참기름막국수!!!!!!!! 정말 맛있습니다. 막국수 별로 안좋아해서,, ...
5        박경호  4달 전  아주 맛있는 국수명인이 운영하는 가게\n막국수 맛있고 만두도 맛있어요.\n기본 반찬...
6   MIN JUNG  6달 전  김치만두가 너무너무 먹고싶어서, 집근처 맛집을 알아보던중 뱅뱅막국수에 방문했습니다....
7   Jisu Yoo  8달 전  사장님과 직원분들이 너무 친절하시고, 참기름막국수, 샤브샤브, 메밀전, 육전 주문했...
8        김우현  5달 전  음식 명장 & 명인 만든 음식이라고 기대를 많이 해서 그런지 물국수는 평범했음\n참...
9        박상욱  7달 전  봉평면메밀촌 시절부터 비빔막국수에 대한 극찬을 한 적이 있는데 이 집이 이렇게나 유...


In [62]:
driver.quit()

In [65]:
reviews_df['Review'][0]

'막국수의 대가🌟🌟🌟🌟🌟\n배가 작은거에 한탄하며 모든 음식을 먹지 못해 슬플 정도.\n들기름 막국수에 충격먹었어요.\n들기름 막국수 감탄하며 먹다보면 참기름 막국수가 생각나고...\n참기름 막국수 맛있게 먹고나면 코다리 막국수가 생각나고....코다리 막국수 먹으면 들기름 막국수가 생각나는 무한굴레🤦\u200d♂️\n깨까지 갈아먹으면 정말 고소하고 더 맛있습니다.\n플레이팅까지 아름다운데 맛까지 맛있네요.\n육전이랑 손만두까지 정말 맛있어요. 손만두가 진짜 맛있었는데 요즘은 있는지 모르겠네요ㅠ\n내부 인테리어도 취저입니다. 주차는 주차장이 있지만 자리가 없는 경우도 있어요..'

In [70]:
driver.quit()

In [71]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 구글 맵 페이지 열기 및 '뱅뱅막국수' 검색
driver.get("https://www.google.com/maps")
try:
    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)

    # 검색 결과가 나타날 때까지 대기
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")

try:
    # 리뷰 페이지로 이동
    next_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    next_button.click()

    # 리뷰 섹션 로드 대기
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
    )
    print('리뷰로 넘어감')
except (NoSuchElementException, TimeoutException) as e:
    print(f"리뷰 페이지로 이동하는 중 오류 발생: {e}")

# 스크롤 다운하여 모든 리뷰 로드
def scroll_down():
    try:
        scrollable_div = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="feed"]'))
        )
    except TimeoutException:
        print("스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.")
        return

    last_review_count = 0
    while True:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)  # 충분한 대기 시간

        # 새롭게 로드된 리뷰 수 확인
        reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
        new_review_count = len(reviews)
        
        if new_review_count >= 728 or new_review_count == last_review_count:
            break  # 필요한 리뷰 수를 확보했거나 더 이상 리뷰가 로드되지 않을 때 종료
        last_review_count = new_review_count

scroll_down()

# 리뷰 데이터 수집
reviews_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
for review in reviews:
    try:
        # '자세히' 버튼 클릭
        try:
            more_button = WebDriverWait(review, 5).until(
                EC.element_to_be_clickable((By.XPATH, './/span[2]/button'))
            )
            more_button.click()
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.wiI7pd'))
            )  # '자세히' 클릭 후 리뷰 로드 대기
        except NoSuchElementException:
            pass  # '자세히' 버튼이 없으면 무시하고 진행
        
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
        reviews_list.append({
            "ID": reviewer_id,
            "Date": date,
            "Review": review_text
        })
    except Exception as e:
        print(f"리뷰 수집 중 오류: {e}")
        continue

# 데이터프레임으로 변환 및 출력
reviews_df = pd.DataFrame(reviews_list)
print(reviews_df)

# CSV 파일로 저장
# reviews_df.to_csv('bangbang_makguksu_reviews.csv', index=False, encoding='utf-8-sig')

# 드라이버 종료
# driver.quit()


리뷰 페이지로 이동하는 중 오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x00D53913+24035]
	(No symbol) [0x00CDBCA4]
	(No symbol) [0x00BBC2D3]
	(No symbol) [0x00BFDC86]
	(No symbol) [0x00BFDECB]
	(No symbol) [0x00C3B9D2]
	(No symbol) [0x00C1FED4]
	(No symbol) [0x00C3953F]
	(No symbol) [0x00C1FC26]
	(No symbol) [0x00BF218C]
	(No symbol) [0x00BF310D]
	GetHandleVerifier [0x00FF9743+2800659]
	GetHandleVerifier [0x010542FE+3172302]
	GetHandleVerifier [0x0104CF12+3142626]
	GetHandleVerifier [0x00DF6CC0+692624]
	(No symbol) [0x00CE4CBD]
	(No symbol) [0x00CE19C8]
	(No symbol) [0x00CE1B60]
	(No symbol) [0x00CD4010]
	BaseThreadInitThunk [0x76C5FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x76F3809E+286]
	RtlGetAppContainerNamedObjectPath [0x76F3806E+238]

스크롤 가능한 리뷰 섹션을 찾을 수 없습니다.
          ID  Date                                             Review
0      Amy J  2주 전  막국수의 대가🌟🌟🌟🌟🌟\n배가 작은거에 한탄하며 모든 음식을 먹지 못해 슬플 정도....
1  Sujin Kim  3달 전  들기름막국수, 참기름막국수, 코다리막국수를 먹었는데 3개 메뉴 모두 맛있었어요. 국...
2        정의진  4달 

# 되는거

In [ ]:
import re 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, WebDriverException
import time

#ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

#ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

#구글 맵 페이지 열기 및 '뱅뱅막국수' 검색
driver.get("https://www.google.com/maps")
try:
    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("뱅뱅막국수")
    search_box.send_keys(Keys.ENTER)
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

#페이지 로드 대기
time.sleep(5)
#리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except Exception as e:
    print(f"리뷰 페이지를 열 수 없습니다: {e}")
    driver.quit()
    exit()


#리뷰 섹션의 컨테이너를 찾음
try:
    review_container = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde'))  # 리뷰 컨테이너의 CSS 선택자
    )

#스크롤을 반복적으로 내리기
    while True:
        # 현재 로드된 리뷰 수 확인
        reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
        previous_count = len(reviews)

#스크롤을 아래로 내림
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_container)
        time.sleep(5)  # 스크롤 후 로딩 대기

#스크롤 후 로드된 리뷰 수 다시 확인
        reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
        current_count = len(reviews)

#새로운 리뷰가 로드되지 않았으면 종료
        if current_count == previous_count:
            print("더 이상 로드할 리뷰가 없습니다.")
            break

    print("모든 리뷰를 끝까지 로드했습니다.")
except Exception as e:
    print(f"스크롤 중 오류 발생: {e}")
#리뷰 데이터 수집
reviews_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
for review in reviews:
    try:
        # '자세히' 버튼 클릭
        try:
            more_button = review.find_element(By.XPATH, './/span[2]/button')
            more_button.click()
            time.sleep(1)  # '자세히' 클릭 후 리뷰 로드 대기
        except NoSuchElementException:
            pass  # '자세히' 버튼이 없으면 무시하고 진행

#리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
        reviews_list.append({
            "ID": reviewer_id,
            "Date": date,
            "Review": review_text
        })
    except Exception as e:
        print(f"리뷰 수집 중 오류: {e}")
        continue

#데이터프레임으로 변환 및 출력
reviews_df = pd.DataFrame(reviews_list)
print(reviews_df)

#CSV 파일로 저장
#reviews_df.to_csv('bangbang_makguksu_reviews.csv', index=False, encoding='utf-8-sig')

드라이버 종료
#driver.quit()